In [1]:
import numpy as np
import pandas as pd
import gffpandas.gffpandas as gffpd

In [2]:
## GRCh37, gencode, Release 44 
## Comprehensive gene annotation
## https://ftp.ebi.ac.uk/pub/databases/gencode/Gencode_human/release_44/GRCh37_mapping/gencode.v44lift37.annotation.gtf.gz
## https://ftp.ebi.ac.uk/pub/databases/gencode/Gencode_human/release_44/GRCh37_mapping/gencode.v44lift37.annotation.gff3.gz
gencode_df = gffpd.read_gff3("gencode.v44lift37.annotation.gff3").df

In [3]:
## HGNC: HUGO Gene Nomenclature Committee
## https://www.genenames.org/download/archive/#!/#tocAnchor-1-2
## https://ftp.ebi.ac.uk/pub/databases/genenames/hgnc/tsv/hgnc_complete_set.txt
hgnc_df        = pd.read_csv("hgnc_complete_set.txt", sep = "\t", header=0, keep_default_na=False)
hgnc_symbol_df = hgnc_df[["hgnc_id", "symbol", "name", "alias_symbol", "alias_name", "prev_symbol", "prev_name"]]

In [4]:
## list of gene names extracted from condon_by_gene.txt.gz
cbase_gene_df = pd.read_table("gene_list.txt", header = None, names = ["xxx", "gene"])

In [5]:
def get_gencode_attr_value(x, key):
    for pair in x.split(";"):
        (k,v) = pair.split("=")
        if k == key:
            return v

In [6]:
gencode_df = gencode_df[gencode_df.type == "exon"]
gencode_df["gene_name"] = gencode_df.attributes.apply(lambda x: get_gencode_attr_value(x, "gene_name"))
gencode_df["exon_number"]  = gencode_df.attributes.apply(lambda x: get_gencode_attr_value(x, "exon_number"))
gencode_df["exon_id"]      = gencode_df.attributes.apply(lambda x: get_gencode_attr_value(x, "ID"))
gencode_df["exon_group_id"]= gencode_df["exon_id"].apply(lambda x: x.split(":")[1])

In [7]:
import warnings

# ["hgnc_id", "symbol","name","alias_symbol","alias_name","prev_symbol","prev_name"]

def is_symbol(gene):
    rec = hgnc_symbol_df[hgnc_symbol_df.symbol == gene]
    if len(rec) > 0 :
        return True
    return False

def convert2symbol(gene):
    for symbol in ["alias_symbol","prev_symbol","name","alias_name","prev_name"]:
        rec = hgnc_symbol_df[hgnc_symbol_df[symbol].apply(lambda x: True if x.split("|").count(gene) > 0 else False)]
        if len(rec) > 0 :
            return rec.symbol.iloc[0]
            
    warnings.warn("Symbol %s is not matched in HGNC!"%gene, ResourceWarning)
    return None

In [8]:
def assemble_into_bed_format(chm:str, start:int, start_ext:int, end:int, end_ext:int, group_size:int, 
                             index:int, name:str, score:str, strand:str):
    ## bed format: 
    ## Chromosome: (e.g. chr3, chrY, chr2_random) or scaffold (e.g. scaffold10671) name
    ## chromStart: Start coordinate on the chromosome or scaffold for the sequence considered (the first base on the chromosome is numbered 0)
    ## chromEnd:   End coordinate on the chromosome or scaffold for the sequence considered. This position is non-inclusive, unlike chromStart.
    ## group_size: total number of exons of the transcription
    ## index: index number of exons in the transcription
    ## name: Name of the line in the BED file
    ## score: Score between 0 and 1000
    ## strand: DNA strand orientation (positive ["+"] or negative ["-"] or "." if no strand)
    return [chm.replace("chr",""), start, start_ext, end, end_ext, group_size, index, name, score, strand]

def assemble_target_region_by(gene_name):
    gene_exons_df = gencode_df[gencode_df.gene_name == gene_name]
    if len(gene_exons_df) > 0 :
        # chose the group which has max number of exons
        exon_group_id  = gene_exons_df.groupby("exon_group_id").size().idxmax()
        exon_group_size= gene_exons_df.groupby("exon_group_id").size().max()
        gene_exons_df  = gene_exons_df[gene_exons_df.exon_group_id == exon_group_id]
        
        # name: "gene_name:exons group size:exon number"
        return pd.DataFrame([assemble_into_bed_format(row.seq_id,row.start,row.start,row.end,row.end,exon_group_size,int(row.exon_number),
                                    "%s:%d:%d"%(row.gene_name,exon_group_size,int(row.exon_number)),row.score,row.strand) 
                                    for ind,row in gene_exons_df.iterrows()],
                             columns = ["Chromosome", "chromStart", "chromStart_ext", "chromEnd", "chromEnd_ext", 
                                        "exon_group_size", "exon_number", "gene_name", "score", "strand"])
        
    else:
        return pd.DataFrame([], columns = ["Chromosome", "chromStart", "chromStart_ext", "chromEnd", "chromEnd_ext", 
                                           "exon_group_size", "exon_number", "gene_name", "score", "strand"])
    
def write_bed_dataframe(fd, bed_df):
    for ind,row in bed_df.iterrows():
        fd.write("%s\t%d\t%d\t%s\t%s\t%s\n"%(row.Chromosome, row.chromStart_ext, row.chromEnd_ext, row.gene_name, row.score, row.strand))


In [9]:
gene_missmatch_list = []

def ext_chromStart_splice_sites(x):
    if x.exon_number == 1:
        return x.chromStart - 1
    else:
        # extend the start to splice sites if it is NOT the first codon
        return x.chromStart - 1 - 2
    
def ext_chromEnd_splice_sites(x):
    if x.exon_number == x.exon_group_size:
        return x.chromEnd + 1
    else:
        # extend the end to splice sites if it is NOT the last codon
        return x.chromEnd + 1 + 2

with open("genes.bed", "w") as fd:
    for gene_name in cbase_gene_df.gene:
    
        bed_df = assemble_target_region_by(gene_name)
        bed_df["chromStart_ext"] = bed_df.apply(ext_chromStart_splice_sites, axis = 1)
        bed_df["chromEnd_ext"]   = bed_df.apply(ext_chromEnd_splice_sites, axis = 1)
        
        if len(bed_df) > 0:
            write_bed_dataframe(fd, bed_df)
            continue
            
        if not is_symbol(gene_name): 
            # convert gene name to stardard symbol
            symbol = convert2symbol(gene_name)
            bed_df   = assemble_target_region_by(symbol)
            if len(bed_df) > 0:
                write_bed_dataframe(fd, bed_df)
                continue
        
        # gene_name in genocode is not standard symbol
        # TODO: match all the possible symbols
        # TODO: raise warning and correct HGNC(HUGO Gene Nomenclature Committee)
        # warnings.warn("gene name [%s] is mismatched in HGNC!\n"%gene_name)
        gene_missmatch_list.append(gene_name)

#### execute the following command line on the server: 

```
[j_wang@flamingo-lg-01 Workspace]$ bedtools getfasta -fi /mnt/beegfs/userdata/a_ivashkin/references/genome_data/gatk/human_g1k_v37.fasta -bed genes.bed -name -tab -fo gencode_codons.tsv
```

In [10]:
import os

# os.system('bedtools getfasta -fi /mnt/beegfs/userdata/a_ivashkin/references/genome_data/gatk/human_g1k_v37.fasta -bed genes.bed -name -tab -fo gencode_codons.tsv')

In [11]:
triplets = ["AAA", "AAC", "AAG", "AAT", "CAA", "CAC", "CAG", "CAT",
            "GAA", "GAC", "GAG", "GAT", "TAA", "TAC", "TAG", "TAT", 
            "ACA", "ACC", "ACG", "ACT", "CCA", "CCC", "CCG", "CCT", 
            "GCA", "GCC", "GCG", "GCT", "TCA", "TCC", "TCG", "TCT", 
            "AGA", "AGC", "AGG", "AGT", "CGA", "CGC", "CGG", "CGT", 
            "GGA", "GGC", "GGG", "GGT", "TGA", "TGC", "TGG", "TGT", 
            "ATA", "ATC", "ATG", "ATT", "CTA", "CTC", "CTG", "CTT", 
            "GTA", "GTC", "GTG", "GTT", "TTA", "TTC", "TTG", "TTT"]

In [12]:
codons_df = pd.read_csv("gencode_codons.tsv", header=None, names=["gene_code","exons_seq"], sep='\t')

In [13]:
codons_df["gene_name"]       = codons_df.gene_code.apply(lambda x: x.split(":")[0])
codons_df["exon_group_size"] = codons_df.gene_code.apply(lambda x: x.split(":")[1])
codons_df["exon_group_id"]   = codons_df.gene_code.apply(lambda x: x.split(":")[2])

In [14]:
## problem: splice sites may be located in a codon. 
## solution: store splice sites and exons seperately 

def get_splice_sites_context(gene_group): 
    
    splice_sites = []
    
    for ind, row in gene_group.sort_values(['exon_group_id'],ascending=True).iterrows(): 
        if row.exon_group_id == 1:
            # splice site of first exon 
            splice_site_context_1 = triplets.index(row.exons_seq[-5:-2])
            splice_site_context_2 = triplets.index(row.exons_seq[-4:-1])
            splice_sites.append([splice_site_context_1, splice_site_context_2])
            
        elif row.exon_group_id == row.exon_group_size:
            # splice site of last exon 
            splice_site_context_1 = triplets.index(row.exons_seq[0:3])
            splice_site_context_2 = triplets.index(row.exons_seq[1:4])
            splice_sites.append([splice_site_context_1, splice_site_context_2])
        
        else:
            # splice site on the left of exon
            splice_site_context_1 = triplets.index(row.exons_seq[0:3])
            splice_site_context_2 = triplets.index(row.exons_seq[1:4])
            splice_sites.append([splice_site_context_1, splice_site_context_2])
            
            # splice site on the right of exon
            splice_site_context_1 = triplets.index(row.exons_seq[-5:-2])
            splice_site_context_2 = triplets.index(row.exons_seq[-4:-1])
            splice_sites.append([splice_site_context_1, splice_site_context_2])
            
    return splice_sites
    
def get_exons_context(gene_group): 
    
    exons = []
    
    for ind, row in gene_group.sort_values(['exon_group_id'],ascending=True).iterrows():
        if row.exon_group_id == 1 :
            # first exon
            exon_seq = row.exons_seq[:-2]
            exons = exons + [ triplets.index(exon_seq[i,i+3]) for i in range(len(exon_seq)-2) ]
            
        elif row.exon_group_id == row.exon_group_size :
            # last exon 
            exon_seq = row.exons_seq[2:]
            exons = exons + [ triplets.index(exon_seq[i,i+3]) for i in range(len(exon_seq)-2) ]
            
        else:
            exon_seq = row.exons_seq[2:-2]
            exons = exons + [ triplets.index(exon_seq[i,i+3]) for i in range(len(exon_seq)-2) ]
    
    if (len(exons_seq)) % 3 != 0:
        raise ValueError("%s has only %d nucleotide!"%(exons_seq[1:-1], len(exons_seq)))
        
    exons = [[triplets.index(exons[i  ,i+3]), triplets.index(exons[i+1,i+4]), triplets.index(exons[i+2,i+5])]
                for i in range(0,len(exons_seq),3)]
    
    return exons

with open("gencode_codons_by_gene.txt", "w") as fd:
    
    for name,group in codons_df.groupby("gene_name"):
        splice_sites = get_splice_sites_context(group)
        exons          = get_exons_context(group)
        
        fd.write("gene\t%s\n"%name)
        
        for ss in splice_sites:
            fd.write("%d\t%d\n"%(ss[0],ss[1]))
            
        for ex in exons:
            fd.write("%d\t%d\t%d\n"%(ex[0],ex[1],ex[2]))
            

TypeError: string indices must be integers

In [16]:
for name,group in codons_df.groupby("gene_name"):
    for idx, row in group.sort_values(['exon_group_id'],ascending=True).iterrows():
        print(idx, row)
        
    break

179272 gene_code                             A1BG:8:1::19:58864769-58864861
exons_seq          CCCACAGCAAGAGAAAGACCACGAGCATGGACATGATGGTCGCGCT...
gene_name                                                       A1BG
exon_group_size                                                    8
exon_group_id                                                      1
Name: 179272, dtype: object
179273 gene_code                     A1BG:8:2::19:58864655-58864696
exons_seq          ACATATGGCTGCTTCTGTCACTGGGCCCCAGGTGACACCTG
gene_name                                               A1BG
exon_group_size                                            8
exon_group_id                                              2
Name: 179273, dtype: object
179274 gene_code                             A1BG:8:3::19:58864291-58864566
exons_seq          ACTTGGCCCTGTCAGCTCCAGGAGCTTGCTCAGCTGGGTCCATCCT...
gene_name                                                       A1BG
exon_group_size                                                   